# Copernicus Global Land Cover data extract and download
## Troms og Finnmark
### Mosses and lichens, grass, shrubs and trees

In [ ]:
!date

In [ ]:
import pkg_resources
if not any(list(map(lambda i: i.key == 'openeo', pkg_resources.working_set))):
    !pip install openeo

In [ ]:
# data
import numpy as np #arrays
import pandas as pd #dataframes
import xarray as xr #multidimensional arrays
import geopandas as gpd #geospatial vector data
import openeo
import zarr

# Regridding <- xesmf is not available in ppc64el pangeo-notebook, yet
import xesmf as xe

# Image processing
from skimage.io import imshow
from skimage.morphology import erosion, dilation

# Visualization
import matplotlib
import matplotlib.pyplot as plt
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
from bokeh.models.formatters import DatetimeTickFormatter
from holoviews import opts

# If running on a local machine execute the following cell instead to create a local dask cluster, otherwise skip it

In [ ]:
# Create a local dask cluster on the local machine
from dask.distributed import Client
with Client(n_workers=24, memory_limit='2GiB', threads_per_worker=1) as client:
    print(client)

In [ ]:
# client.close()

In [ ]:
# Define the Area Of Interest
AOI_name = 'Troms-Finnmark'

In [ ]:
# Get information about the area of interest from the Global Administrative Unit Layers GAUL G2015_2014 
try:
    AOI = gpd.read_file('../data/Troms-Finnmark.geojson')
    type(AOI)
except:
    GAUL = gpd.read_file('zip+https://mars.jrc.ec.europa.eu/asap/files/gaul1_asap.zip') 
    AOI = GAUL[(GAUL.name1 == 'Troms') | (GAUL.name1 == 'Finnmark')] 
    AOI.to_file('../data/Troms-Finnmark.geojson', driver='GeoJSON') 

In [ ]:
# Exract the polygon geometry
AOI_poly = AOI.geometry
type(AOI_poly)

In [ ]:
# Plot the AOI polygon
AOI_poly.plot()

In [ ]:
# Define the minimum and maximum latitude and longitudes for the AOI
AOI_Troms = AOI_poly.bounds.values.tolist()[0]
AOI_Finnmark = AOI_poly.bounds.values.tolist()[1]
AOI_min_lon = min(AOI_Troms[0], AOI_Finnmark[0])
AOI_max_lon = max(AOI_Troms[2], AOI_Finnmark[2])
AOI_min_lat = min(AOI_Troms[1], AOI_Finnmark[1])
AOI_max_lat = max(AOI_Troms[3], AOI_Finnmark[3])
print(AOI_min_lon, AOI_max_lon, AOI_min_lat, AOI_max_lat)

## If the Global Land Cover data has already been downloaded then skip the steps below until when the data is available locally

In [ ]:
session = openeo.connect("https://openeo.vito.be").authenticate_oidc(provider_id="egi")

In [ ]:
session.list_collections()

## Global Land Cover
Global Land Cover products at 100 m resolution are delivered annually by the global component of the Copernicus Land Service. The most recent collection 3 (version 3.0.1) of 100 m Land Cover products for the years 2015 - 2019 were generated from the PROBA-V 100 m and 300 m satellite observations and several other ancillary datasets with global coverage. These Land Cover products provide a main discrete land cover classification map according to UN-FAO Land Cover Classification System LCCS. Additional continuous fractional layers for all basic land cover classes which give the percentage of a 100 m pixel that is filled with a specific land cover class, are also included in the Land Cover products to provide more detailed information on each land cover class.

In [ ]:
session.describe_collection('GLOBAL_LAND_COVER')

In [ ]:
# All the bands/variables
datacube = session.load_collection(
    'GLOBAL_LAND_COVER',
    spatial_extent={"west": AOI_min_lon, "south":AOI_min_lat , "east": AOI_max_lon, "north":AOI_max_lat },
    temporal_extent = ["2015-01-01", "2019-01-01"],
)

In [ ]:
# Download as netCDF into the local ../data folder
# datacube.download(f"../data/C_GlobalLandCover_20150101_20190101_Troms-Finnmark.nc", format="NetCDF")

## Once the **Global Land Cover** data is available locally in ../data/C_GlobalLandCover_20150101_20190101_Troms-Finnmark.nc

### Read the data from local netCDF file

In [ ]:
# Read data from local file and make smaller chunks (less than 1 MiB each)
# Rename dimensions as lat/lon and time
# Extract DataArray with Area of Interest from the Dataset as Variables_AOI
# (this automatically closes the dataset after use)
with xr.open_dataset(f'../data/C_GlobalLandCover_20150101_20190101_Troms-Finnmark.nc', engine ="netcdf4", 
                     chunks={"t": 1, "x": 16708/4, "y": -1}) as cglc_ds:
    cglc_ds = cglc_ds.rename(x='lon', y='lat', t='time')
    Variables_AOI = cglc_ds.sel(lat=slice(AOI_max_lat, AOI_min_lat), lon=slice(AOI_min_lon, AOI_max_lon))

In [ ]:
Variables_AOI

In [ ]:
# Trigger the actual loading of this dataset's data from disk into memory
Variables_AOI.compute()

In [ ]:
# Set a EPSG:4326 Geodetic coordinate reference system
Variables_AOI.rio.write_crs(4326, inplace=True)

In [ ]:
# Clip the data with the polygon that has been obtained for Troms & Finnmark
Variables_AOI = Variables_AOI.rio.clip(AOI_poly, crs=4326)

In [ ]:
Variables_AOI

In [ ]:
# Drop variables not directly of interest here
Variables_AOI = Variables_AOI.drop_vars(['crs',
                                       'Bare_CoverFraction_layer',
                                       'Discrete_Classification_map', 
                                       'Discrete_Classification_proba',
                                       'Forest_Type_layer',
                                       'Snow_CoverFraction_layer',
                                       'BuiltUp_CoverFraction_layer',
                                       'PermanentWater_CoverFraction_layer',
                                       'SeasonalWater_CoverFraction_layer',
                                       'DataDensityIndicator',
                                       'Change_Confidence_layer',
                                       'dataMask'])

In [ ]:
Variables_AOI

In [ ]:
# Calculate the sum over lattitude and longitude
stats = Variables_AOI.sum(dim=['lat', 'lon'])

In [ ]:
%%time
stats.compute()

In [ ]:
# Rename remaining variables (for convenience)
stats = stats.rename(Crops_CoverFraction_layer = 'Crops',
                     Grass_CoverFraction_layer = 'Grass',
                     MossLichen_CoverFraction_layer = 'MossLichen',
                     Shrub_CoverFraction_layer = 'Shrub',
                     Tree_CoverFraction_layer = 'Tree')

In [ ]:
stats.to_dataframe()

In [ ]:
# Calculate total fractional coverage (for all vegetation species), for each year
total = stats.Crops + stats.Grass + stats.MossLichen + stats.Shrub + stats.Tree

In [ ]:
total.data

In [ ]:
# Express the fractional cover in terms of percentage of the total
fractional_cover_in_percentage_of_total = stats.to_dataframe().div(total).mul(100)

In [ ]:
fractional_cover_in_percentage_of_total

In [ ]:
# Plot fraction covers for crops, grass, moss-lichen, shrub and tree 
# and only show the year as xlabel
# also add title
formatter = DatetimeTickFormatter(years=["%Y"], months=["%Y"])
plot = fractional_cover_in_percentage_of_total.hvplot(cmap="RdYlGn", 
                                                      width=1000, height=500, 
                                                      xformatter=formatter)
(plot.relabel('GC3-SC1 "Arctic browning and winter warming" in Troms and Finnmark (Norway)\n(data from the Copernicus Land Monitoring Service Global Land Cover)'
             ).opts(fontsize={'title': 16, 'labels': 14, 'xticks': 12, 'yticks': 12}
             ).opts(xlabel='Year', ylabel='Fractional cover\n[in % of all vegetation types]'))

# Compared to 2015, the cumulated fractional cover of grass (which represent the vast majority of the vegetation in Troms and Finnmark) has decreased (-12%), those of crops (-5%) and moss/lichen (+2%) are about the same, however there are more trees (+10%) and even more shrubs (+24%).

## As far as trees and shrubs are concerned, it was also concluded in ISBN: 978-9935-431-90-5 that *there has been increased growth and encroachment of shrubs and trees in parts of the low Arctic. Plant abundance remained mainly stable, but when changes occurred, shrubs, mosses and lichens were most affected.*

## That could be also the *result from trampling disturbance, grazing pressure, and the excretion of waste*, as reported for Svalbard reindeer in DOI: 10.1016/j.polar.2013.11.002.

## To assess how moss/lichen evolved in the last 20 years or so, we can have a closer look at NDVI long term series (LTS) as a whole, and since the presence of **trees and shrubs are very likely to dominate the signal** we may have to use a "mask" derived from where pixels were classified as moss/lichen in the period 2015-2019 in the NDVI and/or Global Land Cover data.

## If the **NDVI** data has already been downloaded then skip the steps below until when the data is available locally

In [ ]:
# NDVI
datacube = session.load_collection(
    'CGLS_NDVI_V3_GLOBAL',
    spatial_extent={"west": AOI_min_lon, "south":AOI_min_lat , "east": AOI_max_lon, "north":AOI_max_lat },
    temporal_extent = ["1999-01-01", "2020-06-21"],
)

In [ ]:
datacube.download(f'../data/C_CGLS_NDVI_V3_GLOBAL_1990101_20200621_Troms-Finnmark.nc', format='NetCDF')

## Once the **Global Land Cover** data is available locally in ../data/C_CGLS_NDVI_V3_GLOBAL_1990101_20200621_Troms-Finnmark.nc

In [ ]:
# Read data from local file and make smaller chunks (less than 1 MiB each)
# Rename dimensions as lat/lon and time
# Extract DataArray with Area of Interest from the Dataset
# (this automatically closes the dataset after use)
with xr.open_dataset(f'../data/C_CGLS_NDVI_V3_GLOBAL_1990101_20200621_Troms-Finnmark.nc', engine ="netcdf4", 
                     chunks={"t": 52, "x": -1, "y": -1}) as cglc_ndvi_ds:
    cglc_ndvi_ds = cglc_ndvi_ds.rename(x='lon', y='lat', t='time')
    NDVI_AOI = cglc_ndvi_ds.sel(lat=slice(AOI_max_lat, AOI_min_lat), lon=slice(AOI_min_lon, AOI_max_lon))

In [ ]:
NDVI_AOI

In [ ]:
%%time
NDVI_AOI.compute()

In [ ]:
NDVI_AOI

In [ ]:
# Set a EPSG:4326 Geodetic coordinate reference system
NDVI_AOI.rio.write_crs(4326, inplace=True)

In [ ]:
# Clip the data with the polygon that has been obtained for Troms & Finnmark
NDVI_AOI = NDVI_AOI.rio.clip(AOI_poly, crs=4326)

In [ ]:
# Convert NDVI values [-0.08, +0.92]
NDVI_AOI = NDVI_AOI * (1/250) - 0.08

In [ ]:
%%time
NDVI_AOI.compute()

In [ ]:
# Plot NDVI with a slidder to select the date
# map = NDVI_AOI.hvplot(groupby ='time', cmap="RdYlGn", width=1000, height=500)
# (map.relabel("NDVI"))

## Create a MossLichen or MossLichenGrass mask for further processing

In [ ]:
# Create a mask for locations classified as MossLichen in the Global Land Cover
mask_MossLichen = xr.where(Variables_AOI.MossLichen_CoverFraction_layer > 0, 1, 0)

In [ ]:
%%time
mask_MossLichen.compute()

In [ ]:
# Plot the MossLichen mask with slider for time
# mask_MossLichen.hvplot(groupby='time', cmap="RdYlGn", width=1000, height=500).opts(title="Moss & Lichen")

## Skip the next 3 cells is you do not want to see the MossLichen **and Grass** mask

In [ ]:
# Create a mask for locations classified as MossLichen or Grass in the Global Land Cover
mask_MossLichenGrass = xr.where((Variables_AOI.MossLichen_CoverFraction_layer > 0) | (Variables_AOI.Grass_CoverFraction_layer > 0), 1, 0)

In [ ]:
%%time
mask_MossLichenGrass.compute()

In [ ]:
# Plot the MossLichenGrass mask with slider for time
# mask_MossLichenGrass.hvplot(groupby='time', cmap="RdYlGn", width=1000, height=500).opts(title="Moss & Lichen or Grass")

## There is grass nearly everywhere, so using this mask may not lead to result much different from not using it 
## -> try with the MossLichen mask only
## with pixels classified as MossLichen **at least once between 2015 and 2019**

In [ ]:
# Create a mask for locations classified as MossLichen in the Global Land Cover 
mask_MossLichenOnce = xr.where(Variables_AOI.MossLichen_CoverFraction_layer.sum(dim='time') > 0, 1, 0)

In [ ]:
%%time
mask_MossLichenOnce.compute()

In [ ]:
# Plot the MossLichenOnce mask
# mask_MossLichenOnce.hvplot(cmap="RdYlGn", width=1000, height=500).opts(title='Classified as "Moss & Lichen" at least once between 2015 and 2019')

In [ ]:
# Define pattern for image dilation
cross = np.array([[0,1,0], [1,1,1], [0,1,0]])
imshow(cross, cmap = 'gray');

In [ ]:
mask_MossLichenOnce.sum().compute()

In [ ]:
mask_MossLichenOnce

In [ ]:
imshow(mask_MossLichenOnce.values[1000:, 4000:5000], cmap = 'gray');

In [ ]:
dilated_mask = dilation(mask_MossLichenOnce, cross)
dilated_mask.sum()

In [ ]:
imshow(dilated_mask[1000:, 4000:5000], cmap = 'gray');

In [ ]:
dilated_mask = dilation(dilated_mask, cross)
dilated_mask.sum()

In [ ]:
imshow(dilated_mask[1000:, 4000:5000], cmap = 'gray');

In [ ]:
dilated_mask = dilation(dilated_mask, cross)
dilated_mask.sum()

In [ ]:
imshow(dilated_mask[1000:, 4000:5000], cmap = 'gray');

In [ ]:
eroded_mask = erosion(dilated_mask, cross)
eroded_mask.sum()

In [ ]:
imshow(eroded_mask[1000:, 4000:5000], cmap = 'gray');

In [ ]:
eroded_mask = erosion(eroded_mask, cross)
eroded_mask.sum()

In [ ]:
imshow(eroded_mask[1000:, 4000:5000], cmap = 'gray');

In [ ]:
eroded_mask = erosion(eroded_mask, cross)
eroded_mask.sum()

In [ ]:
imshow(eroded_mask[1000:, 4000:5000], cmap = 'gray');

In [ ]:
# Convert numpy array into an xarray data array
eroded_mask = xr.DataArray(eroded_mask, 
                         coords={'lat': mask_MossLichenOnce.lat,'lon': mask_MossLichenOnce.lon}, 
                        dims=["lat", "lon"])

In [ ]:
eroded_mask

In [ ]:
%time
eroded_mask.to_netcdf('../data/Dilatedx3_Erodedx3_MossLichen_Mask.nc', engine='netcdf4')

## Read the MossLichen mask from local netCDF file

In [ ]:
# Read data from local file 
with xr.open_dataset(f'../data/Dilatedx3_Erodedx3_MossLichen_Mask.nc', engine ="netcdf4",
                    chunks={"lat": -1, "lon": -1}) as mask:
    mask = mask.rename(__xarray_dataarray_variable__='MossLichen_mask')

In [ ]:
mask

In [ ]:
mask.compute()

In [ ]:
# Plot the MossLichenOnce mask dilated x3 times and eroded x3 times
mask.to_array().plot()

In [ ]:
!date

# Make a regridder
## The available algorithms are: bilinear, conservative, nearest_s2d (nearest neighbour source to destination), nearest_d2s (or destination to source) and patch (slowest)

In [ ]:
# Define the output grid as an xarray Dataset with the same lon-lat as NDVI_AOI (and in the same order)
mask_regrid = xr.Dataset({"lon": (["lon"], NDVI_AOI.lon.data), "lat": (["lat"], NDVI_AOI.lat.data)})

In [ ]:
mask_regrid.persist()

### Bilinear regridding algorithm

In [ ]:
regridderB = xe.Regridder(mask, mask_regrid, "bilinear")

In [ ]:
# Save the regridder into a local file netCDF file
regridderB.to_netcdf('../data/regridderB.nc')

In [ ]:
mask

In [ ]:
# Apply the regridder to mask
mask_regriddedB = regridderB(mask)

In [ ]:
mask_regriddedB

In [ ]:
# Plot the regridded mask - Bilinear algorithm
mask_regriddedB['MossLichen_mask'].plot()

### Conservative regridding algorithm

In [ ]:
regridderC = xe.Regridder(mask, mask_regrid, "conservative")

In [ ]:
# Save the regridder into a local file netCDF file
regridderC.to_netcdf('../data/regridderC.nc')

In [ ]:
# Apply the regridder to mask
mask_regriddedC = regridderC(mask)

In [ ]:
mask_regriddedC

In [ ]:
# Plot the regridded mask - Conservative algorithm
mask_regriddedC['MossLichen_mask'].plot()

### Patch regridding algorithm

In [ ]:
regridderP = xe.Regridder(mask, mask_regrid, "patch")

In [ ]:
# Save the regridder into a local file netCDF file
regridderP.to_netcdf('../data/regridderP.nc')

In [ ]:
# Apply the regridder to mask
mask_regriddedP = regridderP(mask)

In [ ]:
# Plot the regridded mask - Patch algorithm
mask_regriddedP['MossLichen_mask'].plot()

## The patch and bilinear algorithms provide similar results whereas the conservative algorithms "looses" a lot of points -> use the bilinear regridding which is faster

In [ ]:
# Set a EPSG:4326 Geodetic coordinate reference system
mask_MossLichenOnce.rio.write_crs(4326, inplace=True)

In [ ]:
# Clip the data with the polygon that has been obtained for Troms & Finnmark
mask_MossLichenOnce = mask_MossLichenOnce.rio.clip(AOI_poly, crs=4326)

In [ ]:
%%time
mask_MossLichenOnce.persist()

# Make a regridder
## The available algorithms are: bilinear, conservative, nearest_s2d (nearest neighbour source to destination), nearest_d2s (or destination to source) and patch (slowest)
### Bilinear regridding algorithm

In [ ]:
# Define the output grid as an xarray Dataset with the same lon-lat as NDVI_AOI (and in the same order)
mask_regrid = xr.Dataset({"lon": (["lon"], NDVI_AOI.lon.data), "lat": (["lat"], NDVI_AOI.lat.data)})

In [ ]:
mask_regrid

In [ ]:
# Note that the Regridder works on xarray.Datasets
regridderB = xe.Regridder(mask_MossLichenOnce.to_dataset(), mask_regrid, "bilinear")

In [ ]:
# Check that the regridder can transform data from the input grid shape to the output grid shape
regridderB

In [ ]:
# It is convenient to save the regridder into a local file netCDF file (making it took over 15mn)
regridderB.to_netcdf('../data/regridderB.nc')

In [ ]:
# Apply the regridder to mask_MossLichenOnce
mask_MossLichenOnce_regridded = regridderB(mask_MossLichenOnce.chunk({'lon':'-1', 'lat':'-1'}))

In [ ]:
mask_MossLichenOnce_regridded

In [ ]:
# Multiplot
Variables_AOI.hvplot(groupby ='time', subplots=True, cmap="RdYlGn", width=1400, height=600)

In [ ]:
# Mosses & lichens
LichenCF_AOI.hvplot(groupby ='time', cmap="RdYlGn", width=1400, height=600)

In [ ]:
# Trees
TreeCF_AOI.hvplot(groupby ='time', cmap="RdYlGn", width=1400, height=600)

In [ ]:
# Mosses & lichens
LichenCF_AOI.hvplot(groupby ='time', cmap="RdYlGn", width=1400, height=600)

In [ ]:
# Shrubs
Plot = ShrubCF_AOI.hvplot(groupby ='time', cmap="RdYlGn", width=1400, height=600)
Plot.opts(opts.Overlay(title='Copernicus Land Monitoring Service!n Global Land Cover!n Troms & Finnmark!n Shrubs'))

In [ ]:
NDVI_AOI[23].hvplot.hist(cmap="RdYlGn",bins=25, width=800, height=700)

In [ ]:
NDVI = cgls_dsr.NDVI * (1/250) - 0.08

In [ ]:
NDVI[23].hvplot.hist(cmap="RdYlGn",bins=25, width=800, height=700)

In [ ]:
NDVI_monthly = NDVI.groupby(NDVI.time.dt.month).mean()

In [ ]:
NDVI_monthly.month

In [ ]:
NDVI_msked = NDVI.where((NDVI >= -0.08) & (NDVI <= 0.92)) # by default where condition is false values are set to NA

In [ ]:
NDVI_msked = NDVI_msked.rio.write_crs(4326)

In [ ]:
NDVI_msked

In [ ]:
# Convert DataArray to Dataset, and float64 to float32
NDVI_msked = NDVI_msked.to_dataset().astype('float32')
NDVI_msked

In [ ]:
NDVI_AOI = NDVI_msked.rio.clip(AOI_poly, crs=4326)

In [ ]:
NDVI_AOI.hvplot(groupby ='time', cmap='RdYlGn', width=1400, height=600).hist()

In [ ]:
NDVI_AOI.sel(lat=69.012222, lon=23.040833, method='nearest').hvplot(ylim=(-0.08, 0.92))

In [ ]:
NDVI_AOI['time']

In [ ]:
# Read LTS from Swift
import s3fs
from datetime import datetime
fs = s3fs.S3FileSystem(anon=True, client_kwargs={'endpoint_url': 'https://object-store.cloud.muni.cz'})
s3path = 's3://foss4g-data/CGLS_LTS_1999_2019/*.nc'
remote_files = fs.glob(s3path)

In [ ]:
remote_files

In [ ]:
def preprocess(ds):
    t = datetime.strptime(ds.attrs['identifier'].split(':')[-1].split('_')[1].replace('1999-', ''), "%Y-%m%d")
    return(ds.assign_coords(time=t).expand_dims('time'))  

In [ ]:
# Iterate through remote_files to create a fileset
fileset = [fs.open(file) for file in remote_files]
NDVI_LTS = xr.open_mfdataset(fileset, preprocess=preprocess, combine='nested', concat_dim=['time'], decode_coords="all")

In [ ]:
NDVI_LTS

In [ ]:
import fsspec
import s3fs
import kerchunk.hdf
import xarray as xr

In [ ]:
fs = s3fs.S3FileSystem(anon=True,
      client_kwargs={
         'endpoint_url': 'https://object-store.cloud.muni.cz'
      })

In [ ]:
remote_filename = 'https://object-store.cloud.muni.cz/swift/v1/foss4g-data/CGLS_LTS_1999_2019/c_gls_NDVI-LTS_1999-2019-1221_GLOBE_VGT-PROBAV_V3.0.1.nc'
with fsspec.open(remote_filename) as inf:
    h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, remote_filename, inline_threshold=100)
    chunk_info = h5chunks.translate()

In [ ]:
LTS = xr.open_mfdataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo": chunk_info,
        },
        "consolidated": False
    }
)
LTS

In [ ]:
fs.ls('foss4g-data/CGLS_LTS_1999_2019/')

In [ ]:
from datetime import datetime

In [ ]:
%%time
s3path = 's3://foss4g-data/CGLS_LTS_1999_2019/c_gls_NDVI-LTS_1999-2019-*.nc'
chunk_info_list = []
time_list = []

for file in fs.glob(s3path):
    url = 'https://object-store.cloud.muni.cz/swift/v1/' + file
    t = datetime.strptime(file.split('/')[-1].split('_')[3].replace('1999-', ''), "%Y-%m%d")
    time_list.append(t)
    print('working on ', file)
    with fsspec.open(url) as inf:
        h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, url, inline_threshold=100)
        chunk_info_list.append(h5chunks.translate())

In [ ]:
%%time
from kerchunk.combine import MultiZarrToZarr
mzz = MultiZarrToZarr(
    chunk_info_list,
    coo_map={'INDEX': 'INDEX'},
    identical_dims=['crs'],
    concat_dims=["INDEX"],
)

out = mzz.translate()

In [ ]:
%%time
LTS = xr.open_mfdataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo": out,
        },
        "consolidated": False
    }
)
LTS

In [ ]:
import json

In [ ]:
jsonfile='test.json'
with open(jsonfile, mode='w') as f :
    json.dump(out, f)

In [ ]:
import xarray as xr
LTS = xr.open_mfdataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo":'./test.json',
        },
        "consolidated": False
    }
)
LTS

In [ ]:
catalogue="https://object-store.cloud.muni.cz/swift/v1/foss4g-catalogue/c_gls_NDVI-LTS_1999-2019.json"
NDVI_LTS = xr.open_mfdataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo":catalogue
                    },
        "consolidated": False
    }
)
NDVI_LTS

In [ ]:
LTS = NDVI_LTS.rio.write_crs(4326)

In [ ]:
LTS = LTS.sel(lat=slice(AOI_max_lat,AOI_min_lat) ,lon=slice(AOI_min_lon,AOI_max_lon))

In [ ]:
LTS = LTS.rio.clip(AOI_poly, crs=4326)

In [ ]:
LTS

In [ ]:
#LTS = LTS.chunk({'time':-1, 'lat':'5MB', 'lon':'5MB'})

In [ ]:
LTS['min'][5].hvplot(geo=True)

In [ ]:
NDVI_LTS.to_zarr(rf'./data/c_gls_NDVI-LTS_1999-2019_Troms-Finnmark_VGT-PROBAV_V3.zarr', mode='w', )

In [ ]:
# Store zarr is /data/jwd/main/048/543/48543996/working/jupyter/data
LTS = xr.open_zarr(rf'./data/c_gls_NDVI-LTS_1999-2019_Troms-Finnmark_VGT-PROBAV_V3.zarr')

In [ ]:
LTS

In [ ]:
# Continue from here

In [ ]:
LTS['time']

In [ ]:
dates_2022 = pd.date_range('20220101', '20221231')

In [ ]:
decadie = dates_2022[np.isin(dates_2022.day, [1,11,21])]
decadie

In [ ]:
LTS = LTS.assign_coords(time=decadie)
LTS

In [ ]:
# Change dates from 2019 to 2022
#LTS = LTS.assign_coords(time = LTS.time + np.timedelta64(1096, 'D'))

In [ ]:
LTS['max'][20].hvplot(width=1400, height=600)

In [ ]:
# Kautokeino
POI_lat = 69.01894772737091
POI_lon = 23.046880518262814
# Karasjok
#POI_lat = 69.47223
#POI_lon = 25.51885

In [ ]:
LTS_NDVI_POI = LTS.sel(lat=POI_lat, lon=POI_lon, method='nearest')

In [ ]:
LTS_NDVI_POI['mean'].values

In [ ]:
POI_min = LTS_NDVI_POI['min']
POI_max = LTS_NDVI_POI['max']
POI_std = LTS_NDVI_POI['stdev']
POI_mean = LTS_NDVI_POI['mean']
POI = NDVI_AOI.sel(lat=POI_lat, lon=POI_lon, method='nearest')

In [ ]:
fig = plt.figure(1, figsize=[15, 5])
ax=fig.add_subplot(111)
p = {}
for stat in ['max', 'min', 'mean', 'median']:
    p[stat] = LTS_NDVI_POI[stat].sel(time=slice('2022-01-01', '2022-12-31')).plot(ax=ax)

ps = POI.plot(ax=ax, color='green', linestyle='dashed', linewidth=4)
ax.legend((p['max'], p['min'], p['mean'], p['median'], ps) , labels=('Max', 'Min', 'Mean', 'Median', 'NDVI'), 
                                                             loc = 'upper left')
ax.set_ylim([0, 1])
ax.set_title("")

plt.title("NDVI values in 2022 compared to statistics for the period 1999-2019")

In [ ]:
(POI.hvplot(width=1400, height=700, label='NDVI') *
 POI_mean.hvplot(c='grey', label='mean LTS 1999-2019', alpha=0.2, line_dash='dashed') *
 POI_min.hvplot(c='b',label='min LTS 1999-2019', alpha=0.2, line_dash='dashed') *
 POI_max.hvplot(c='r', label='Max LTS 1999-2019', alpha=0.2, line_dash='dashed') *
 hv.Area((POI_mean.time, POI_mean - POI_std, POI_mean + POI_std ), vdims=['- Std', '+ Std'], label='Standard deviation LTS 1999-2019',).opts( color='blue', alpha=0.2))\
    .opts(title=f"CGLS S3 300m NDVI fluctuation over the year 2022 in comparison with CGLS Long Term Statistics\nPoint of interest Lat {np.round(POI.lat.data,2)} Lon {np.round(POI.lon.data,2)}",
          legend_position='bottom_right')

In [ ]:
LTS = LTS.rio.write_crs(4326)

In [ ]:
LTS = LTS.rio.clip(AOI_poly, crs=4326)

# Regrid from 300m to 1km (i.e. LTS resolution)

In [ ]:
NDVI_1k = NDVI_AOI.rio.reproject_match(LTS)

In [ ]:
NDVI_1k.coords

In [ ]:
# Latitude (and perhaps also longitude) values are slightly different between the two datasets -> reassign to avoid issues
NDVI_1k = NDVI_1k.assign_coords(lat=LTS.lat, lon=LTS.lon)
NDVI_1k

In [ ]:
NDVI_1k.coords

In [ ]:
NDVI_1k.hvplot(groupby='time', cmap='RdYlGn', width=1400, height=600)

## The Vegetation COndition Index (VCI) compares the current NDVI to the range of values observed in the same period in previous years

In [ ]:
VCI = ((NDVI_1k - LTS['min']) / (LTS['max'] - LTS['min'])) * 100
VCI.where (VCI > 100) == 100
VCI.where (VCI < 0) == 0

In [ ]:
VCI.name = 'VCI'
VCI

In [ ]:
VCI.hvplot(groupby='time', cmap='RdYlGn', width=1400, height=600)

In [ ]:
VCI.sel(lat=POI_lat, lon=POI_lon, method='nearest')

In [ ]:
VCI.sel(lat=POI_lat, lon=POI_lon, method='nearest').hvplot.hist()

In [ ]:
VCI.plot()

In [ ]:
VCI.isel(time=-1).hvplot.contourf(cmap='RdYlGn', alpha=0.7,
                                  levels=[-200,-100,-50, 0,50,100,200],
                                  geo=True, tiles= 'CartoLight',
                                  width=1400, height=600,
                                  title=f'CGLS VCI Troms-Finnmark {VCI.isel(time=-1).time.dt.date.data}')